In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.style.use('ggplot')
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ParameterGrid
from sklearn.inspection import permutation_importance

In [2]:
##LECTURA DEL FICHERO SPSS Y SUS METADATOS
##Debe estar instalado el paquete de pyreadstat que permite leer ficheros SPSS y SAS
import pyreadstat

df, meta = pyreadstat.pyreadstat.read_sav('D:\TFM\Cuestionarios\CuestionariosEsp\Estudiantes\estudiantesESP1.sav', metadataonly=True)
df2, meta2 = pyreadstat.pyreadstat.read_sav('D:\TFM\Cuestionarios\CuestionariosEsp\Escuelas\Limpieza\escuelasESP1.sav', metadataonly=True)
df3, meta3 = pyreadstat.pyreadstat.read_sav('D:\TFM\Cuestionarios\CuestionariosEsp\profesores\Limpieza\profesoresESP1.sav', metadataonly=True)

In [3]:
pd.options.display.max_colwidth = 500
pd.options.display.max_rows = 999
pd.options.display.max_columns = 9999

In [4]:
data = pd.read_csv('EstuTranTar.csv')

In [5]:
data['ST004D01T']

0        2.0
1        1.0
2        1.0
3        2.0
4        1.0
        ... 
35938    1.0
35939    1.0
35940    1.0
35941    2.0
35942    1.0
Name: ST004D01T, Length: 35943, dtype: float64

In [6]:
data

Unnamed: 0    CNTSCHID    CNTSTUID  LECTURA1    MATES1  CIENCIA1  \
0               0  72400001.0  72400490.0  0.586109  0.635132  0.576692   
1               1  72400001.0  72401482.0  0.744381  0.752894  0.759850   
2               2  72400001.0  72402362.0  0.832005  0.857496  0.796113   
3               3  72400001.0  72402959.0  0.514129  0.560212  0.540197   
4               4  72400001.0  72403316.0  0.748630  0.720412  0.757500   
...           ...         ...         ...       ...       ...       ...   
35938       35938  72401102.0  72421796.0  0.678839  0.694708  0.717064   
35939       35939  72401102.0  72425905.0  0.830551  0.826280  0.847201   
35940       35940  72401102.0  72426602.0  0.733722  0.749129  0.717696   
35941       35941  72401102.0  72444251.0  0.830822  0.863035  0.820881   
35942       35942  72401102.0  72446056.0  0.801112  0.822784  0.791520   

        GLOBAL1  LECTURA2    MATES2  CIENCIA2   GLOBAL2  ST001D01T_7.0  \
0      0.609538  0.596666  0.609567  0.601336  0.607698              0   
1      0.715705  0.749909  0.752305  0.758363  0.751188              0   
2      0.827226  0.836496  0.843236  0.836332  0.842870              0   
3      0.552421  0.528629  0.541695  0.540430  0.542919              0   
4      0.679666  0.746114  0.738673  0.751130  0.738106              0   
...         ...       ...       ...       ...       ...            ...   
35938  0.668215  0.688511  0.693371  0.701855  0.693659              0   
35939  0.799105  0.834995  0.834609  0.844056  0.836343              0   
35940  0.693980  0.735940  0.739085  0.737609  0.734141              0   
35941  0.838623  0.839750  0.848758  0.845455  0.849479              0   
35942  0.759658  0.805100  0.809777  0.808700  0.803790              0   

       ST001D01T_8.0  ST001D01T_9.0  ST001D01T_10.0  ST001D01T_11.0  \
0                  0              0               1               0   
1                  0              0               1               0   
2                  0              0               1               0   
3                  0              0               1               0   
4                  0              0               1               0   
...              ...            ...             ...             ...   
35938              0              0               1               0   
35939              0              0               1               0   
35940              0              0               1               0   
35941              0              0               1               0   
35942              0              0               1               0   

       ST003D02T  ST004D01T  ST005Q01TA_1.0  ST005Q01TA_2.0  ST005Q01TA_3.0  \
0            9.0        2.0               1               0               0   
1            8.0        1.0               1               0               0   
2           11.0        1.0               1               0               0   
3            5.0        2.0               0               0               1   
4            2.0        1.0               1               0               0   
...          ...        ...             ...             ...             ...   
35938        3.0        1.0               1               0               0   
35939        7.0        1.0               1               0               0   
35940       10.0        1.0               0               1               0   
35941        5.0        2.0               1               0               0   
35942        2.0        1.0               1               0               0   

       ST005Q01TA_4.0  ST005Q01TA_5.0  ST007Q01TA_1.0  ST007Q01TA_2.0  \
0                   0               0               1               0   
1                   0               0               0               0   
2                   0               0               0               1   
3                   0               0               1               0   
4                   0               0             

In [7]:
#Se borran las columnas que no van a entrar al modelo
## Tambien Eliminamos la altura y el peso, ya que da demasiada información
dataMod = data.drop(['Unnamed: 0','CNTSCHID','CNTSTUID','LECTURA1','MATES1','CIENCIA1','GLOBAL1','LECTURA2','MATES2','CIENCIA2','GLOBAL2','ST004D01T','WB151Q01HA','WB152Q01HA','ST003D02T'],axis=1)

In [8]:
# training /test split
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeClassifier
std_sc = StandardScaler()
x = std_sc.fit_transform(dataMod.values)
y = data['ST004D01T'].values

testsize = 0.2 # in the [0,1] range. 1: 100%
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=testsize, random_state=1)

In [9]:
from sklearn.ensemble import RandomForestClassifier

std_sc = StandardScaler()
x = std_sc.fit_transform(dataMod.values)
#x = dataMod.values



lr_m =  RidgeClassifier(alpha=0.134)

#ajuste sobre la muestra completa, sin split train-test
lr_m.fit(X_train, y_train)



RidgeClassifier(alpha=0.134)

# Validación del modelo con los datos de test

In [10]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

y_pred = np.clip(lr_m.predict(X_test), 0,  100.)



In [11]:
mat_confusion = confusion_matrix(
                    y_true    = y_test,
                    y_pred    = y_pred
                )

accuracy = accuracy_score(
            y_true    = y_test,
            y_pred    =y_pred,
            normalize = True
           )

print("Matriz de confusión")
print("-------------------")
print(mat_confusion)
print("")
print(f"El accuracy de test es: {100 * accuracy} %")
    

Matriz de confusión
-------------------
[[3094  516]
 [ 451 3128]]

El accuracy de test es: 86.54889414383085 %


In [12]:
#Dataframe con el coeficiente y nombre columna
VarCoef = pd.DataFrame(lr_m.coef_.T,dataMod.columns.to_list()).sort_values(0)
VarCoef ['Cols']=VarCoef.index

#Dataframe con las descripciones (convertimos un diccionario a un DataFrame)
Des1 = pd.DataFrame([[key, meta.column_names_to_labels[key]] for key in meta.column_names_to_labels.keys()], columns=['Cols', 'Desc'])
Des2 = pd.DataFrame([[key, meta2.column_names_to_labels[key]] for key in meta2.column_names_to_labels.keys()], columns=['Cols', 'Desc'])
Des3 = pd.DataFrame([[key, meta3.column_names_to_labels[key]] for key in meta3.column_names_to_labels.keys()], columns=['Cols', 'Desc'])
Des = pd.concat([Des1,Des2, Des3],axis=0)


#Hacemos todos los cruces, para que crucen todas las columnas
VarCoef0 = pd.merge(left=Des,right=VarCoef, left_on='Cols', right_on='Cols')

VarCoef0["Coefi_abs"] = abs(VarCoef0[0])
VarImp = VarCoef0.sort_values("Coefi_abs", ascending=False)
VarImp

,Cols,Desc,0,Coefi_abs
16,ST019AQ01T,In what country were you and your parents born? You,-0.030669,0.030669
7,ST011Q08TA,In your home: Books of poetry,0.021812,0.021812
15,ST011D19TA,In your home: <Country-specific wealth item 3>,0.019987,0.019987
12,ST011Q16NA,"In your home: Books on art, music, or design",0.016297,0.016297
24,ST061Q01NA,"How many minutes, on average, are there in a <class period>?",0.015506,0.015506
25,EC158Q01HA,"On the most recent day you attended school, how long did you study in the morning before going to school? Hours",-0.015297,0.015297
21,ST059Q03TA,Typically required to attend: Number of <class periods> per week in <science>,0.011807,0.011807
23,ST060Q01NA,"In a normal, full week at school, how many <class periods> are you required to attend in total?",-0.011398,0.011398
8,ST011Q09TA,In your home: Works of art (e.g. paintings),-0.010460,0.010460
11,ST011Q12TA,In your home: A dictionary,0.009758,0.009758


In [13]:
# select alpha in ridge regression by CV
# First iteration to find alpha
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score, KFold

# define folds
n_folds = 10
kf = KFold(n_folds, shuffle=True)

# define param_grid
l_alpha = [2.**k for k in range(-6, 10)]
param_grid = {'alpha': l_alpha}

# define grid search
lr_m = RidgeClassifier()
ridge_alpha_search = GridSearchCV(lr_m, 
                                  param_grid=param_grid, 
                                  cv=kf, 
                                  scoring='neg_mean_absolute_error',
                                  n_jobs=2, 
                                  verbose=1)

_ = ridge_alpha_search.fit(x, y)

Fitting 10 folds for each of 16 candidates, totalling 160 fits


PicklingError: Could not pickle the task to send it to the workers.

In [ ]:
print("alpha_range: %.2f - %.2f" % (np.array(l_alpha).min(), np.array(l_alpha).max()))
print('alpha = %.2f' % (ridge_alpha_search.best_params_['alpha']))
print('best_cv_mae = %.3f' % (-ridge_alpha_search.best_score_))

plt.title("CV MAEs vs alpha")
plt.xticks(range(len(l_alpha)), l_alpha, rotation=45)
_ = plt.plot( -ridge_alpha_search.cv_results_['mean_test_score'])